<a href="https://colab.research.google.com/github/MinseokKim0813/Travel-Budget-Planner/blob/main/Travel_Budget_Planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This data feature calculates estimated budget for the travel. It checks the hotel price of the city entered by the user, and converts that to the local currency.

API Use:
1. Hotel search API ("Amadeus")
2. Currency API ("Currency Beacon")

In [ ]:
from google.colab import userdata

# API keys
CURRENCY_API_KEY = userdata.get('CurrencyBeacon')
HOTEL_API_KEY = userdata.get('Amadeus')
HOTEL_API_SECRET = userdata.get('AmadeusSecret')

# Or dereference lines below to use my API keys

# CURRENCY_API_KEY = "pwahYg9b6bnequhzEK0qgb1ovob89Un3"
# HOTEL_API_KEY = "GWSN8cJGYUx3Qpf6EnVSJQ0EFKbCorIy"
# HOTEL_API_SECRET = "vArB9kLtJsIqOAUz"


# Error handling
if not CURRENCY_API_KEY:
  print("Err: Currency API key not found")
elif not HOTEL_API_KEY or not HOTEL_API_SECRET:
  print("Err: Hotel API key not found")




Getting hotel prices near the specific city using Amadeus API

Documentation link: https://developers.amadeus.com/self-service/category/hotels/api-doc/hotel-list/api-reference

1. Getting Access Token

In [ ]:
import requests

def get_access_token(api_key, api_secret):
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "client_id": api_key,
        "client_secret": api_secret
    }

    # Make the request to get the token
    response = requests.post(url, headers=headers, data=data)


    # If the response is successful (200 OK), extract the token
    if response.status_code == 200:
        return response.json().get("access_token", "No access token found in response")
    else:
        return f"Error: {response.status_code}, {response.text}"



api_key = HOTEL_API_KEY
api_secret = HOTEL_API_SECRET
token = get_access_token(api_key, api_secret)



2. Getting hotel list in the city

Link for reference: https://developers.amadeus.com/self-service/category/hotels/api-doc/hotel-list/api-reference

Testing:

I have tested with 3 different cities: SEL (Seoul), NYC (New York City), PAR (Paris). And refined the code so it works for all of them. So the user can get budget plan for different cities.

API rate limit consideration:

10 transactions per second,
1 request / 100ms

So I'll make it wait for 110 ms after request

API Rate Limit Reference Link: https://developers.amadeus.com/pricing

In [ ]:

# Get access token again
api_key = HOTEL_API_KEY
api_secret = HOTEL_API_SECRET
token = get_access_token(api_key, api_secret)

import time

def search_hotels_by_city(city_code, radius, radius_unit, token):
    url = f"https://test.api.amadeus.com/v1/reference-data/locations/hotels/by-city?cityCode={city_code}&radius={radius}&radiusUnit={radius_unit}&hotelSource=ALL"
    headers = {"Authorization": f"Bearer {token}"}

    # Make the request
    response = requests.get(url, headers=headers)

    # Print status code and response for debugging
    print(f"Status Code: {response.status_code}")

    # API Limit Consideration:
    time.sleep(0.11)

    # Error Handling:
    # Check if the response is successful and return hotel data
    if response.status_code == 200:
        return response.json()  # Return the hotel data as JSON
    else:
        return f"Error: {response.status_code}, {response.text}"

# Example usage
city_code = "PAR"  # City code can be found here: https://www.iata.org/en/publications/directories/code-search/?airport.search=SEL
radius = 5  # Radius in KM
radius_unit = "KM"
hotel_results = search_hotels_by_city(city_code, radius, radius_unit, token)


Status Code: 200


We only need hotel ID, and since it is showing too many hotels so only show 10 hotels

In [ ]:
# store the hotel ID, 10 hotels
hotel_ID_array = []
for hotelNum in range (10):
  hotel_ID = hotel_results['data'][hotelNum]['hotelId']
  hotel_ID_array.append(hotel_ID)

# check if successfully stored
for id in hotel_ID_array:
  print(id)

YXPARRND
HNPARKGU
HNPARNUJ
HNPARSPC
BWPAR599
BWPAR196
RTPARKKK
OIPAR555
BLPARY3E
RTPAR157


The list only showed the name, address, hotelID and not the prices. So call the API again with each hotel ID for the price.

3. Search for the hotel prices

In [ ]:
# get access token again in case it already expired
token = get_access_token(api_key, api_secret)


# get hotel details
def get_hotel_details(hotel_id, token, check_in_date, adult_num = 1, room_quantity = 1):
    # Construct the URL for the hotel offers search
    url = f"https://test.api.amadeus.com/v3/shopping/hotel-offers?hotelIds={hotel_id}&adults={adult_num}&checkInDate={check_in_date}&roomQuantity={room_quantity}&paymentPolicy=NONE&bestRateOnly=false"
    headers = {"Authorization": f"Bearer {token}"}

    # Make the request to get hotel offers
    response = requests.get(url, headers=headers)


    # Hotel availability
    global available
    available = False

    # API Limit Consideration:
    time.sleep(0.11)

    # Error Handling:
    # I found that it gives error when there are no available rooms for the hotel, or when the hotel data is outdated
    # If the hotel is available
    if response.status_code == 200:
        # Return price of the hotel
        available = True
        return response.json()["data"][0]["offers"][0]["price"]


# Hotel search details
check_in_date = "2024-10-30"
adult_num = 1
room_quantity = 1

price_array = []
counter = 1

# Loop through the hotel IDs and get offer price for each hotel
for hotel_id in hotel_ID_array:
    hotel_offers = get_hotel_details(hotel_id, token, check_in_date, adult_num, room_quantity)

    key_array = []

    # Only include ones available (unavailable ones or outdated ones cause Error)
    # And get the prices of the available hotels
    if available:

      # I noticed that each city has different JSON offer format (the key string is different; sometimes "base" and sometimes "total")
      # So I checked the keys of the JSON first and checked the appropriate key
      if "total" in hotel_offers.keys():
        price_array.append(hotel_offers["total"])
      elif "base" in hotel_offers.keys():
        price_array.append(hotel_offers["base"])

      currency = hotel_offers["currency"]

    print(str(counter) + " hotels processed")
    counter += 1



# Get the average price
ave_price = 0
for each_price in price_array:
  ave_price += float(each_price)

ave_price /= len(price_array)
ave_price = int(ave_price)

print("Average prices, Currency: " + str(ave_price) + ", " + currency)




1 hotels processed
2 hotels processed
3 hotels processed
4 hotels processed
5 hotels processed
6 hotels processed
7 hotels processed
8 hotels processed
9 hotels processed
10 hotels processed
Average prices, Currency: 37, EUR


Now I'm gonna implement currency API to convert the average price into the currency I want.

API Documentation link: https://currencybeacon.com/api-documentation

Currency codes: https://currencybeacon.com/supported-currencies

API Restriction Consideration:
In terms of service document (at section 4.2), it mentions that I should not store the data for more than 8 hours. In my case, I am only using the currency exchange rate for immediate calculation and do not store the currency rate, so I am using it appropriately. There is no other restriction or quota limit.

Terms of Service Link: https://currencybeacon.com/terms

In [ ]:
# Convert currency
def convert_currency(base_currency, converted_currency):
  url = f"https://api.currencybeacon.com/v1/latest?api_key={CURRENCY_API_KEY}&base={base_currency}&symbols={converted_currency}"
  response = requests.get(url)

  if response.status_code == 200:
    # Get the currency exchange rate
    currency_rate = response.json()["response"]["rates"][converted_currency]
    # Calculate the price
    converted_price = ave_price * currency_rate
    print("For " + str(adult_num) + " adult, on " + check_in_date + ", for " + str(room_quantity) + " room, in " + city_code + " requires at least:")
    print(str(ave_price), str(base_currency) + " = " + str(converted_price), converted_currency)
  else:
    print("Error: " + response.status_code)


# Base currency is the original currency for the found hotels
base_currency = currency
# Use currency code for the desired currency
converted_currency = "USD"

convert_currency(base_currency, converted_currency)

For 1 adult, on 2024-10-30, for 1 room, in PAR requires at least:
37 EUR = 40.83693145 USD
